# Tutorial 2a: Tidy data and split-apply-combine
(c) 2017 Justin Bois. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT).

In [64]:
import numpy as np
import pandas as pd

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

Loading BokehJS ...

Note: * The code in this tutorial comes from the tutorial by Justin Bois, which can be found [here](http://bebi103.caltech.edu.s3-website-us-east-1.amazonaws.com/2017/tutorials/t2a_tidy_data.html)
## Introduction
The data we are investigating come from David Prober's lab. A description of their work on the genetic regulation of sleep can be found on the research page of the lab website. There is a movie of the moving/sleeping larvae similar to the one used to produce the data set we are using in this tutorial. The work based on this data set was published in (Gandhi et al., *Neuron*, 85, 1193–1199, 2015).

## The genotype data
First, we will load the genotype file.

In [65]:
with open('data/130315_1A_genotypes.txt', 'r') as f:
    for _ in range(30):
        print(next(f), end='')

# Genotype data from the Gandhi, et al. experiment ending March 13, 2013
#
# The experiment was performed in a 96 well plate with zebrafish
# embryos.  Gene sequencing was later used to identify the genotype
# of the fish in each well.  Not all fish could be genotyped.
#
# The mutants being studied have deletions in the gene coding for
# arylalkylamine N-acetyltransferase (aanat), which is a key enzyme
# in the rhythmic production of melatonin.  Melatonin is a hormone
# responsible for regulation of circadian rhythms.  It is often taken
# as a drug to treat sleep disorders.  The goal of this study is to
# investigate the effects of aanat deletion on sleep pattern in
# 5+ day old zebrafish larvae.
#
# Each column lists the wells corresponding to the genotype of
# each fish.  If a number is missing (between 1 and 96), the 
# genotype of that fish is not known.
# 
# These data were kindly provided by Avni Gandhi and Audrey Chen
# from David Prober's lab.  They were part of the paper Gandh

Each column in this file contains a list of wells in the 96-well plate corresponding to each genotype. Apparently, the columns are tab delimited. We can parse this using the `delimiter` keyword argument of `pd.read_csv()`. When specifying delimiters, tabs are denoted as `\t`. 
<br>
<br>
There are two header rows, one starting with `Genotype 1` and the other one starting with `WH 17`. The first one is really redundant, so it can be skipped. Nevertheless, we will read them both in as headers using the `header` kwarg of `pd.read_csv()`.

In [66]:
# Load in the genotype file, call it df_gt for genotype DataFrame
df_gt = pd.read_csv('data/130315_1A_genotypes.txt',
                    delimiter='\t', 
                    comment='#',
                    header=[0 ,1])

# Let's look at it
df_gt

,Genotype1,Genotype2,Genotype3
,WT 17,Het 34,Mut 22
0,2.0,1,4.0
1,14.0,3,11.0
2,18.0,5,12.0
3,24.0,6,13.0
4,28.0,8,20.0
5,29.0,10,21.0
6,30.0,15,23.0
7,54.0,19,27.0
8,58.0,22,35.0


Now we have two header rows for the columns. If we look at the column names, we see that they are a `MultiIndex` instance.

In [67]:
df_gt.columns

MultiIndex(levels=[['Genotype1', 'Genotype2', 'Genotype3'], ['Het 34', 'Mut 22', 'WT 17']],
           labels=[[0, 1, 2], [2, 0, 1]])

This is part of Pandas's cool multi-indexing functionality, which is not comething necessary if we have tidy data. However, the messay data with multi-indexing may result in performance boosts for accessing data. This can be important for very large data sets. But in general, tidy data are easier to conceptualize and syntactically much simpler to access.
<br>
<br>
In this case, we do not need the multi-indexing, and zero-level indexing (`Genotype1`, etc.), so we will leave just level one index. This can be accomplished using the `get_level_values` method of Pandas `MultiIndex` objects.

In [68]:
# Reset the columns to be the second level of indexing
df_gt.columns = df_gt.columns.get_level_values(1)

# Check out the new columns
df_gt.columns

# Let's actually clean up the column names
df_gt.columns = ['wt', 'het', 'mut']

The above approach to renaming is a shortcut to using `rename()` method that has previously been encountered. It could actually have been done before messing around with the multi-indexing, but why not to learn something new? We need to know how to tidy data sets in this format anyways.

## Tidying the genotype data
As they are, the data are not tidy. For tidy data, we would have two columns, `location`, which gives the well number for each larva, and `genotype`, which is `wt` for wild type, `het` for heterozygote, or `mut` for mutant.
<br>
<br>
A useful data tidying tool is the `pd.melt()` function. For this simple data set, it takes the column headings and makes them into a column (with repeated entries) and puts the data accordingly in the correct order. We just need to specify the name of the "variable" column, in this case the genotype, and the name of the "value" column, in this case, the fish ID.

In [69]:
# Tidy the DataFrame
df_gt = pd.melt(df_gt, var_name='genotype', value_name='location')

# Take a look
df_gt

,genotype,location
0,wt,2.0
1,wt,14.0
2,wt,18.0
3,wt,24.0
4,wt,28.0
5,wt,29.0
6,wt,30.0
7,wt,54.0
8,wt,58.0
9,wt,61.0


We still have some unhelpful `NaN` entries, so we drop them using `dropna()` method of `DataFrame`s.

In [70]:
# Drop NaNs
df_gt = df_gt.dropna()

# Take a look
df_gt

,genotype,location
0,wt,2.0
1,wt,14.0
2,wt,18.0
3,wt,24.0
4,wt,28.0
5,wt,29.0
6,wt,30.0
7,wt,54.0
8,wt,58.0
9,wt,61.0


We can see that indices have some skips due to using the NaN's. This is not really of concern, since we will not use them. We can use the `reset_index()` to reset the indexing of the `DataFrame`.

In [71]:
# Reset the indexing of the DataFrame
df_gt = df_gt.reset_index(drop=True)

We now have a tidy `DataFrame`. However, the fish numbers are floates. This happens becuase NaN is a float, so the column got converted to floats. We can reset the data type of the column to `int`s.

In [72]:
# Set data types to be integers
df_gt.loc[:, 'location'] = df_gt.loc[:, 'location'].astype(int)

We now have a beautiful, tidy `DataFrame` of genotypes. Apparently, the `pd.melt()` may be the most useful function in tidying messy data sets. Now let's take a look at the behavioral data.

## The behavioral data
We will first take a look at the behavioral data set. It has been preprocessed by Justin Bois (thanks, Justin!)  to some degree, and we will work with the original raw data later on. Let's look at the contents of the data file.

In [73]:
with open('data/130315_1A_aanat2.csv', 'r') as f:
    for _ in range(40):
        print(next(f), end='')

# Lightly processed data from VideoTracker from the Gandhi, et al. experiment
# concluding on March 15, 2013.
#
# The experiment was performed in a 96 well plate with zebrafish
# embryos. Gene sequencing was later used to identify the genotype
# of the fish in each well. Not all fish could be genotyped.
#
# The mutants being studied have deletions in the gene coding for
# arylalkylamine N-acetyltransferase (aanat), which is a key enzyme
# in the rhythmic production of melatonin. Melatonin is a hormone
# responsible for regulation of circadian rhythms. It is often taken
# as a drug to treat sleep disorders. The goal of this study is to
# investigate the effects of aanat deletion on sleep pattern in
# 5+ day old zebrafish embryos.
#
# Activity is defined as the number of seconds over the one-minute interval 
# in which a given larva was moving.
#
# The column 'zeit' contains the so-called Zeitgeber time in units of hours.
# The Zeitgeber time is zero when the lights come on on the first 

The provided data set is tidy. Each row corresponds to the measured seconds of activity of a single fish for a single minute. Other information includes the time of the measurement and the day in the life of the fish. The Zeitgeber time seem to somehow give us a "reference time" for the experiment (`Zeit` ist "time" in Deutsch. Alles klar?)
<br>
<br>
Let's load the data set

In [74]:
df = pd.read_csv('data/130315_1A_aanat2.csv', comment='#')

# Let's take a look
df.head()

,location,activity,time,zeit,zeit_ind,day
0,1,0.6,2013-03-15 18:31:09,-14.480833,-869,4
1,2,1.4,2013-03-15 18:31:09,-14.480833,-869,4
2,3,0.0,2013-03-15 18:31:09,-14.480833,-869,4
3,4,0.0,2013-03-15 18:31:09,-14.480833,-869,4
4,5,0.0,2013-03-15 18:31:09,-14.480833,-869,4


## Adding the genotype information
First of all, we can try to sort out data by including the genotype information for the fish at each location. The `DataFrame` genotyp has two columns, `location` and `genotype`. The `location` column is shared with our behavior `DataFrame` so we can use `pd.merge()` to add the genotype information as a new column.

In [75]:
df = pd.merge(df, df_gt)

# Take a look
df.head()

,location,activity,time,zeit,zeit_ind,day,genotype
0,1,0.6,2013-03-15 18:31:09,-14.480833,-869,4,het
1,1,1.9,2013-03-15 18:32:09,-14.464167,-868,4,het
2,1,1.9,2013-03-15 18:33:09,-14.447500,-867,4,het
3,1,13.4,2013-03-15 18:34:09,-14.430833,-866,4,het
4,1,15.4,2013-03-15 18:35:09,-14.414167,-865,4,het


## Light or dark?
We can attempt to annotate our `DataFrame` to indicate if it is light or dark. Then we can easily separate the diurnal and nocturnal activity. So let's make a column `light` which will have the entry `True` if the lights are on, and `False` if the lights are off. Referring again to the comments in the header of the data set, the light come on at 9AM and turn off at 11PM. We can use the `time` column to determine this.
<br>
We should first look at the time column and determine its data type.

In [76]:
df['time'].dtype

dtype('O')

The result `dtype('O')` means that the data type is "object", which is a generic catch-all for unknown data types. However, we know that the `time` column are actual clock times. We can tell this to Pandas and unleash its data processing power on our data set. We do this using the `pd.to_datetime()` function.

In [77]:
df['time'] = pd.to_datetime(df['time'])

# What is the data type now?
df['time'].dtype

dtype('<M8[ns]')

The data type is `<M8[ns]`, which is essentially saying that Pandas is aware that this is a point in time, and it is stored with nanosecond precision. This has some implications: <br> 
* If data are taken on the greater than nanosecond frequency, Pandas's datetime utility will not help. In such case, we would simply use a number (say femtoseconds) as the time variable.
* The time zero can not be more than about a thousand years ago. This can come up in geology or fields like that. 
<br>
<br>
We can do lots of time-based things with this datetime format, like extraction of the time from the datetime. We use `.dt.time` to do this.

In [78]:
df['time'].dt.time.head()

0    18:31:09
1    18:32:09
2    18:33:09
3    18:34:09
4    18:35:09
Name: time, dtype: object

We can also compare times. For example, let's ask if the time is after 9AM.

In [79]:
(df['time'].dt.time > pd.to_datetime('9:00:00').time()).head()

0    True
1    True
2    True
3    True
4    True
Name: time, dtype: bool

We had to convert the string `9:oo:oo` to datetime and then extract the time using the `time()` method. <br>
Now we can just make a column of booleans reporting whether the time is greater than 9:00:00 and less than 23:00:00.

In [80]:
df['light'] = ( (df['time'].dt.time >= pd.to_datetime('9:00:00').time())
               & (df['time'].dt.time < pd.to_datetime('23:00:00').time()))

# Take a look
df.head()

,location,activity,time,zeit,zeit_ind,day,genotype,light
0,1,0.6,2013-03-15 18:31:09,-14.480833,-869,4,het,True
1,1,1.9,2013-03-15 18:32:09,-14.464167,-868,4,het,True
2,1,1.9,2013-03-15 18:33:09,-14.447500,-867,4,het,True
3,1,13.4,2013-03-15 18:34:09,-14.430833,-866,4,het,True
4,1,15.4,2013-03-15 18:35:09,-14.414167,-865,4,het,True


We make a quick plot of the Zeitgeber time versus light. We only need to do this for a single location (single well).

In [81]:
p = bokeh.plotting.figure(plot_height=259,
                          plot_width=700,
                          x_axis_label='Zeitgeber time (hours)',
                          y_axis_label='light')

inds = df['location'] == 1
p.circle(df.loc[inds, 'zeit'], df.loc[inds, 'light'])

bokeh.io.show(p)

Apparently, the light switch on and off as it should.
## Split-apply-combine
Let's now compute some more things. For instance, let's say we want to compute the average per-minute activity of each fish over the course of the experiment. To do this, we must:
1. **Split** the data set according to the `location` field, i. e. split it up so we have a separate data set for each fish.
2. **Apply** an averaging function to the activity in these split data sets.
3. **Combine** the results of these averages on the split data set into a new, summary data set that contains the locations and means for each location.

The strategy we want is a **split-apply-combine** strategy. This idea was put forward by Hadley Wickham in [this paper](https://www.jstatsoft.org/article/view/v040i01).